In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

# Load Data
Data can be downloaded from [Kaggle](https://www.kaggle.com/datasets/wordsforthewise/lending-club?datasetId=902). Information about the columns can be seen here on [google drive](https://drive.google.com/file/d/1UzyKMtrzL3aEEQpeYIz_qjBWyW-lAMr0/view).

In [10]:
accepted_df = pd.read_csv(filepath_or_buffer="data/raw/accepted_2007_to_2018Q4.csv")
rejected_df = pd.read_csv(filepath_or_buffer="data/raw/rejected_2007_to_2018Q4.csv")

/var/folders/h0/sqy12xn11_gg038734m010lh0000gn/T/ipykernel_64802/15118711.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  accepted_df = pd.read_csv(filepath_or_buffer="data/raw/accepted_2007_to_2018Q4.csv")


In [17]:
# Take on subset of the whole data to better work with it
# TODO: Delete this later for the final fit of our model(s)
# Attention: For NaNs Identification etc... use full dataset
n_sample = 50000
accepted_df = accepted_df.sample(n=n_sample)
rejected_df = rejected_df.sample(n=n_sample)

In [18]:
# Print information about raw datasets
print(f"Shape of accepted_df: {accepted_df.shape}")
print(f"Shape of rejected_df: {rejected_df.shape}")

Shape of accepted_df: (50000, 151)
Shape of rejected_df: (50000, 9)


In [19]:
accepted_df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
100908,62317158,NaN,10000.0,10000.0,10000.0,60 months,10.99,217.38,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1520079,134444734,NaN,16500.0,16500.0,16500.0,60 months,12.13,368.12,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
634633,113196313,NaN,15000.0,15000.0,15000.0,36 months,7.21,464.60,A,A3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1637029,589858,NaN,5000.0,5000.0,5000.0,60 months,13.61,115.34,C,C2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
362167,41450845,NaN,8000.0,8000.0,8000.0,36 months,9.17,255.04,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning
In this section we remove features that contain too many NaNs and which are also not (or less) relevant for our prediction.